In [1]:
import pybamm as pb;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False
import openpyxl
import traceback
import multiprocessing

In [2]:
# Add funtion to sys.path
os.chdir(pb.__path__[0]+'/..')   
# pb.__path__[0] allow us to location path without concerning different machine
import sys  
sys.path.append(os.path.join(os.getcwd(),'wip\Rio_Code\Fun_P2'))  
# sys.path.append(os.path.join(pb.__path__[0]+'/..','wip\Rio_Code\Fun_P2')) 
# wip\Rio_Code\Fun_P2\Fun_P2_Crack_simp.py 
from Fun_P2_Union_crack import (
    GetScan,
    Run_model_w_dry_out ,
    Run_model_wo_Dry_out,
    Cal_new_con_Update,
    Run_Model_Base_On_Last_Solution,
    Run_Model_Base_On_Last_Solution_RPT,
    write_excel_xlsx,)

In [3]:
# Test this branch - Coupled_RioP2 newly created from Simon's develop branch
# Run without dry-out model
V_max = 4.2;        V_min = 2.5;   Temper_RPT = 25; 
Total_Cycles = 10; save_at_cycles = 1;  
for i in range(0,1):  # Key scan parameters:
    Diff_SEI_i = 1.7e-22; 
    R_SEI_i = 2E5;   Bulk_Sol_Con_i = 4541.0;
    D_Li_inSEI_i = 1e-12;    # default: 1e-20 
    c_Li_inte_ref_i = 15;    # default: 15
    Diff_EC_i   = 2e-20;       k_SEI_i   = 1e-16;
    LAMcr_prop_i=2.7778e-08;   Crack_rate_i=3.9e-20; 
    Couple_SEI_LiP_i = 1e-6; # default: 1e-6
    k_LiP_i = 1E-10;         # default: 1e-10
    Temper = 25;
    mesh_par = 30;
     
for i in range(0,1):  
    ChemistryChen=pb.parameter_sets.OKane2022   
    #ChemistryChen["electrolyte"] = "lipf6_Valoen2005"
    Para_0=pb.ParameterValues(chemistry=ChemistryChen)
    # DFN parameter
    Para_0.update({"Upper voltage cut-off [V]": 4.21})
    Para_0.update({"Lower voltage cut-off [V]": 2.49})
    Para_0.update({"Negative electrode diffusivity [m2.s-1]": 2e-13})
    Para_0.update({"Positive electrode diffusivity [m2.s-1]": 1e-13})
    # general SEI
    Para_0.update({'SEI resistivity [Ohm.m]':R_SEI_i}) ;
    # solvent-diffusion limited
    Para_0.update({'Outer SEI solvent diffusivity [m2.s-1]':Diff_SEI_i});
    Para_0.update({'Bulk solvent concentration [mol.m-3]':Bulk_Sol_Con_i});
    # interstitial-diffusion limited
    Para_0.update({'Inner SEI lithium interstitial diffusivity [m2.s-1]':D_Li_inSEI_i})       
    Para_0.update({'Lithium interstitial reference concentration [mol.m-3]':c_Li_inte_ref_i}) 
    # ec-reaction limited
    Para_0.update({'EC diffusivity [m2.s-1]':Diff_EC_i}) 
    Para_0.update({'SEI kinetic rate constant [m.s-1]':k_SEI_i}) 
    Para_0.update({'EC initial concentration in electrolyte [mol.m-3]':Bulk_Sol_Con_i}) 
    # LiP and coupling with SEI:
    Para_0.update({"Dead lithium decay constant [s-1]": Couple_SEI_LiP_i})   
    Para_0.update({'Lithium plating kinetic rate constant [m.s-1]':k_LiP_i})  
    # Crack model
    Para_0.update({"Negative electrode LAM constant proportional term [s-1]": LAMcr_prop_i})
    Para_0.update({"Positive electrode LAM constant proportional term [s-1]": LAMcr_prop_i})  
    # make it simple for now, but may want to have T dependency in the future
    Para_0.update({"Negative electrode cracking rate": Crack_rate_i})
    Para_0.update({"Positive electrode cracking rate": Crack_rate_i})
    Para_0.update({"Negative electrode volume change": 0.0})
    Para_0.update({"Positive electrode volume change": 0.0}) 
    Model_0 = pb.lithium_ion.DFN(     ) #
    var = pb.standard_spatial_vars  
    var_pts = {
        var.x_n: 20,  
        var.x_s: 10,  
        var.x_p: 20,  
        var.r_n: mesh_par,  
        var.r_p: mesh_par,  }

In [4]:
# Run break-in cycle step by step:

Experiment_Breakin_0 = pb.Experiment( [
    ( f"Discharge at 1C until {V_min} V",  
    "Rest for 1 hours",  
    f"Charge at 0.3C until {V_max} V" )  ] *3 )
# Run 1st cycle
Sim_Breakin_0    = pb.Simulation(
    Model_0,        experiment = Experiment_Breakin_0,
    parameter_values = Para_0,
    solver = pb.CasadiSolver(),
    var_pts=var_pts,
    ) 
Sol_Breakin_0    = Sim_Breakin_0.solve(calc_esoh=False)

In [83]:
def GetSol_dict (keys_all, Sol_Breakin_0, cycle_no, step_CD ,  step_CC  ):
    my_dict = {}; # initialize my_dict based on 
    for keys in keys_all:
        for key in keys:
            my_dict[key]=[];
    [keys_loc,keys_tim,keys_cyc]  = keys_all;
    # get time_based variables:
    if len(keys_tim): 
        for key in keys_tim:
            if key[0:2] == "CD":
                step_no = step_CD;
                if key[3:] == "Time [h]":
                    my_dict[key].append  (
                        Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[3:]].entries
                        -
                        Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[3:]].entries[0])
                else:
                    my_dict[key].append  (
                        Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[3:]].entries)
            elif key[0:2] == "CC":
                if key[3:] == "Time [h]":
                    my_dict[key].append  (
                        Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[3:]].entries
                        -
                        Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[3:]].entries[0])
                else:
                    my_dict[key].append  (
                        Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[3:]].entries)
    # get cycle_step_based variables:
    if len(keys_cyc): 
        for key in keys_cyc:
            if key in ["Discharge capacity [A.h]"]:
                step_no = step_CD;
                my_dict[key].append  (
                    Sol_Breakin_0.cycles[cycle_no].steps[step_no][key].entries[-1]
                    - 
                    Sol_Breakin_0.cycles[cycle_no].steps[step_no][key].entries[0])
            elif key[0:5] in ["CDend","CCend","CDsta","CCsta",]:
                if key[0:2] == "CD":
                    step_no = step_CD;
                    if key[2:5] == "sta":
                        my_dict[key].append  (
                            Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[6:]].entries[0])
                    elif key[2:5] == "end":
                        my_dict[key].append  (
                            Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[6:]].entries[-1])
                elif key[0:2] == "CC":
                    step_no = step_CC;
                    if key[2:5] == "sta":
                        my_dict[key].append  (
                            Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[6:]].entries[0])
                    elif key[2:5] == "end":
                        my_dict[key].append  (
                            Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[6:]].entries[-1])

    # get location_based variables:
    if len(keys_loc): 
        for key in keys_loc:
            if key in ["x_n [m]","x [m]","x_s [m]","x_p [m]"]:
                print("These variables only add once")
                if not len(my_dict[key]):   # special: add only once
                    my_dict[key] = Sol_Breakin_0[key].entries[:,-1]
            elif key[0:5] in ["CDend","CCend","CDsta","CCsta",]:       # special: need to start from 0
                print("These variables add multiple times")
                if key[0:2] == "CD":
                    step_no = step_CD;
                    if key[2:5] == "sta":
                        my_dict[key].append  (
                            Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[6:]].entries[:,0])
                    elif key[2:5] == "end":
                        my_dict[key].append  (
                            Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[6:]].entries[:,-1])
                elif key[0:2] == "CC":
                    step_no = step_CC;
                    if key[2:5] == "sta":
                        my_dict[key].append  (
                            Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[6:]].entries[:,0])
                    elif key[2:5] == "end":
                        my_dict[key].append  (
                            Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[6:]].entries[:,-1])                

    return my_dict           

In [86]:
print(keys_all)

[[], ['CD Time [h]', 'CD Terminal voltage [V]'], ['Discharge capacity [A.h]', 'CDend Local ECM resistance [Ohm]', 'CDsta Negative electrode SOC', 'CDend Negative electrode SOC']]


In [84]:
cycle_no = 1; step_CD = 0;  step_CC = 2;
my_dict = GetSol_dict (keys_all, Sol_Breakin_0, cycle_no, step_CD ,  step_CC  )

In [ ]:
print(my_dict)

In [72]:
keys_loc =[];
keys_all = [keys_loc,keys_tim,keys_cyc];
print(keys_all)

[[], ['CD Time [h]', 'CD Terminal voltage [V]'], ['Discharge capacity [A.h]', 'CDend Local ECM resistance [Ohm]', 'CDsta Negative electrode SOC', 'CDend Negative electrode SOC']]


In [68]:
# design the function:
# input: sol; list of varibales that want to save; 
# output: dict of varbales and results which can be saved as .mat;
# define three types of keys: location related; time related; cycle related
# Function input:
cycle_no = 1; step_no = 1;
keys_loc = [
    # Default output:
    "x [m]",
    "x_n [m]",
    "x_s [m]",
    "x_p [m]",
    # default: end; 
    "CCend Porosity",
    "CCend Negative electrode interfacial current density [A.m-2]",
    "CCend Electrolyte potential [V]",
    "CCend Electrolyte concentration [mol.m-3]",
    "CDend Porosity",
    "CDend Negative electrode interfacial current density [A.m-2]",
    "CDend Electrolyte potential [V]",
    "CDend Electrolyte concentration [mol.m-3]",
]
keys_tim = [
    # default: CDend
    "CD Time [h]",

    "CD Terminal voltage [V]",
]
keys_cyc = [   # default: CDend
    "Discharge capacity [A.h]",
    "CDend Local ECM resistance [Ohm]",
    "CDsta Negative electrode SOC", 
    "CDend Negative electrode SOC",
    #"CDend Loss of capacity to lithium plating [A.h]",
    #"CDend Loss of capacity to SEI [A.h]",
    #"CDend Loss of capacity to SEI on cracks [A.h]",
    #"CDend X-averaged total SEI on cracks thickness [m]",
]
keys_all = [keys_loc,keys_tim,keys_cyc];


In [ ]:
# get time_based variables:
cycle_no = 1; step_CD = 0;  step_CC = 2;
for key in keys_tim:
    if key[0:2] == "CD":
        step_no = step_CD;
        if key[3:] == "Time [h]":
            my_dict[key].append  (
                Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[3:]].entries
                -
                Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[3:]].entries[0])
        else:
            my_dict[key].append  (
                Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[3:]].entries)
    elif key[0:2] == "CC":
        if key[3:] == "Time [h]":
            my_dict[key].append  (
                Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[3:]].entries
                -
                Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[3:]].entries[0])
        else:
            my_dict[key].append  (
                Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[3:]].entries)
            

In [57]:
# get cycle_step_based variables:
cycle_no = 1; step_CD = 0;  step_CC = 2;
for key in keys_cyc:
    if key in ["Discharge capacity [A.h]"]:
        step_no = step_CD;
        my_dict[key].append  (
            Sol_Breakin_0.cycles[cycle_no].steps[step_no][key].entries[-1]
            - 
            Sol_Breakin_0.cycles[cycle_no].steps[step_no][key].entries[0])
    elif key[0:5] in ["CDend","CCend","CDsta","CCsta",]:
        if key[0:2] == "CD":
            step_no = step_CD;
            if key[2:5] == "sta":
                my_dict[key].append  (
                    Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[6:]].entries[0])
            elif key[2:5] == "end":
                my_dict[key].append  (
                    Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[6:]].entries[-1])
        elif key[0:2] == "CC":
            step_no = step_CC;
            if key[2:5] == "sta":
                my_dict[key].append  (
                    Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[6:]].entries[0])
            elif key[2:5] == "end":
                my_dict[key].append  (
                    Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[6:]].entries[-1])
            

In [ ]:
# get location_based variables:
cycle_no = 1; step_CD = 0;  step_CC = 2;
for key in keys_loc:
    if key in ["x_n [m]","x [m]","x_s [m]","x_p [m]"]:
        print("These variables only add once")
        if not len(my_dict[key]):   # special: add only once
            my_dict[key] = Sol_Breakin_0[key].entries[:,-1]
    elif key[0:5] in ["CDend","CCend","CDsta","CCsta",]:       # special: need to start from 0
        print("These variables add multiple times")
        if key[0:2] == "CD":
            step_no = step_CD;
            if key[2:5] == "sta":
                my_dict[key].append  (
                    Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[6:]].entries[:,0])
            elif key[2:5] == "end":
                my_dict[key].append  (
                    Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[6:]].entries[:,-1])
        elif key[0:2] == "CC":
            step_no = step_CC;
            if key[2:5] == "sta":
                my_dict[key].append  (
                    Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[6:]].entries[:,0])
            elif key[2:5] == "end":
                my_dict[key].append  (
                    Sol_Breakin_0.cycles[cycle_no].steps[step_no][key[6:]].entries[:,-1])

In [ ]:
my_dict